# Demo for bruk av openAI med cv data fra CV partner

Dette er ment for å være en rask demo for å vise hvordan openai apiet fungerer, hvordan man kan laste inn CV data, og gi alle et godt utgangspunkt for hvordan å jobbe med ChatGPT gjennom et API

In [1]:
import os
from openai import OpenAI
from dotenv import load_dotenv

Her bruker vi noe som heter dotenv for å laste miljø variabler fra en .env fil som ligger i root dir. Dette er en enkel måte å organisere miljøvariabler på per prosjekt, og man slipper all styret man typisk har i de forskjellige operativ systemene (Windows spesielt).

Denne biten med kode antar at det finnes en fil i root med navn `.env`. I den filen bør det være en linje `OPENAI_API_KEY = ...` der man fyller in API nøkkelen. Denne filen ligger i .gitignore så derre må lage den selv.

In [2]:
load_dotenv("../.env")
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

Her har jeg en beskrivelse på en rolle som jeg (Ask) har søkt på som et eksempel for bruk i denne demoen. Her har jeg kopiert beskrivelsen og gjort litt manuell redigering for å fjerne tegn som ikke er standard. Dette er litt prep arbeid før vi tar i bruk ChatGPT.

In [3]:
Rolle_beskrivelse = """1   Navn på oppdrag /
prosjekt
LLM Engineer (mulig teamleder) til nytt KI Team
2   Kort beskrivelse av oppdraget
Sikt etablerer et nytt team som jobber med kunstig intelligens på tvers av virksomheten. Dette
vil støtte produkt-teamene i Sikt med kompetanse og gjennomføringskraft og realisere piloter
på ny og bedre funksjonalitet.
Vi ønsker med dette teamet og etablerere et sterkt fagmiljø for data science og KI i Sikt som
gjør oss i stand til å identifisere, planlegge og utnytte flere av mulighetene som KI åpner for i
produktene våre.
Selv om hovedoppgaven til teamet blir å realisere faktiske produkter og ny funksjonalitet, blir
det også en svært viktig å jobbe sammen med produkt-teamene for å identifisere de riktige
initiativene, samt å dokumentere og formidle både internt og eksternt om både læring, feil og
suksesser.
Vi har identifisert to ulike roller som blir viktig for oss fra starten, og hvor vi søker en erfaren
konsulent. En av disse rollene vil også ha funksjon som teamleder. Til stillingen som «LLM
engineer» («Large Language Model engineer») søker vi en erfaren konsulent med følgende
kvalifikasjoner:
Kvalifikasjoner
(dersom aktuell som teamleder) Erfaring med å lede og inspirere høytytende team
for å fremme et positivt arbeidsmiljø og høy leveransekraft.
\n
Evnen til å veilede og utvikle team-medlemmer, både faglig og personlig, for å styrke
teamets kompetanse og karriereutvikling.
\n
Kommunikasjon: Sterke muntlige og skriftlige kommunikasjonsevner for å forklare
komplekse analyser og konklusjoner til et ikke-teknisk publikum.
\n
Dyptgående programmeringskunnskap, fortrinnsvis Python.
\n
Praktisk erfaring og utforsking av innovativ bruk av LLM via API det siste året.
\n
Kunnskap og erfaring med open source språkmodeller, og fine-tuning av disse.
\n
Erfaring med RAG (Retrieval Augmented Generation) med LangChain eller
tilsvarende.
\n
Grunnleggende forståelse for maskinlæring og NLP (Natural Language Processing)
\n
Erfaring med å utvikle og vedlikeholde API-er.
\n
God forståelse og erfaring med git, CI/CD, skyteknologi og Docker eller kubernetes.
\n
Teamarbeid: Evnen til å jobbe effektivt i tverrfaglige team og samarbeide med andre
fagområder.
\n
Nysgjerrighet og lærevillighet: En evne til å holde seg oppdatert med de siste
trendene og teknologiene innen AI og maskinlæring.
\n
God skriftlig og muntlig fremstillingsevne på både norsk og engelsk
Arbeidsoppgaver:
\n
Sammen med produktteamene idémyldre, identifisere og planlegge nye piloter og
initiativer.
\n
Sammen med teamet utvikle ny funksjonalitet og tjenester med LLM og generativ AI.
\n
Formidling og historiefortelling om arbeidet, prosessen og resultatene underveis
\n
Bidra til godt samarbeid i produktteamet
Personlige egenskaper:
\n
Gode analytiske ferdigheter
\n
Høy gjennomføringsevne
\n
Presis i kommunikasjon
\n
Svært gode samarbeidsevner"""

For å komunnisere med OpenAI API'et så initialiserer vi et client objekt. Dette objektet håndterer hvor kommunikasjon med API'et på et litt høyere nivå.

Det neste vi gjør her er å laste opp min CV i PDF format slik at denne filen er tilgjengelig for clienten å bruke.

Så ber vi clienten å gjøre en chat completion. Dette er metoden får å interagere direkte med en chat. Denne historikken vil ikke bli husket til neste chat completion så hvis man vil ha med historikk så må det være en del av inputet til en slik chat completion.

In [8]:
client = OpenAI()

CV = client.files.create(
  file=open("../data/AskJuhlMarkestad_pdf_cv.pdf", "rb"),
  purpose='assistants'
)

Her gir vi modelen en system melding `"role": "system", "content": ...`, det vil si noe den skal ta med i prompten for hver interaksjon med chaten. I dette tilfellet gir vi den en beskrivelse av hva dens generelle oppgave er. Dette kan også inneholde mer info om hvordan svar skal generelt formateres, eller om f.eks. tone den skal svare med.

Selve meldingen som skal gis kommer under rollen `"role": "user", "content": ...`. Her gir vi mer spesifikke instrukser med noe formatering for å identifisere hva som er CV og hva som er rolle beskrivelse. Her har vi lagt in den spesifikke referansen til den opplatede CV'en.

In [9]:
completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "Du hjelper konsulenter med å spisse deres CV intro text for en gitt rolle beskrivelse."},
    {"role": "user", "content": f"Skriv en CV intro tekst på 150 ord for en gitt CV og en gitt rolle beskrivelse: \n CV: {CV.id} \n Rolle Beskrivelse: {Rolle_beskrivelse}"}
  ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content='Erfaren LLM Engineer med ekspertise innen Large Language Models og teamledelse. Jeg har ledet høytytende team for å skape et positivt arbeidsmiljø og sikre høy leveransekraft. Min evne til å veilede og utvikle medlemmer styrker teamets kompetanse og karrierevekst. Med sterke kommunikasjonsevner formidler jeg komplekse analyser og konklusjoner til et ikke-teknisk publikum. Min dype programmeringskunnskap inkluderer Python, samt praktisk erfaring med innovativ bruk av LLM via API og fine-tuning av open source språkmodeller. Jeg har erfaring med RAG og grunnleggende forståelse for maskinlæring og NLP. Ved å utvikle og vedlikeholde API-er og jobbe effektivt i tverrfaglige team, bidrar jeg til å identifisere, planlegge og realisere nye piloter og initiativer. Gjennom god samarbeidsevne og analytiske ferdigheter leverer jeg presist og effektivt i teammiljøer.', role='assistant', function_call=None, tool_calls=None)


Hver fil vi har lastet opp koster 0.2$ per dag. 

Ved å kjøre koden under så kan man se hvilke files som er lastet opp for et gitt formål, eller totalt sett. Dokumentene vil bli delt med alle som har API tilgang til instansen som laget API nøkkelen. Hver OBS på at man kan laste opp samme fil flere ganger og dermed øke kostnadene. 

Derfor har jeg og lagt ved en liten kodesnutt som sletter alle filene opplastet som eksempel på hvordan man kan rengjøre miljøet sitt.

In [10]:
client.files.list(purpose="assistants")

SyncPage[FileObject](data=[FileObject(id='file-7EAmaQ2d6jDl9aFhWQqsnh2I', bytes=173730, created_at=1712659662, filename='AskJuhlMarkestad_pdf_cv.pdf', object='file', purpose='assistants', status='processed', status_details=None)], object='list', has_more=False)

In [11]:
for file in client.files.list(purpose="assistants"):
    client.files.delete(file.id)
client.files.list(purpose="assistants")

SyncPage[FileObject](data=[], object='list', has_more=False)